In [1]:
import ee
import geemap
import os
import datetime

In [2]:
service_account = 'promedio-a--de-nieve-6bbfb64d1@ee-ezequiellagos.iam.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, 'ee-ezequiellagos-0e8769baf6cb.json')
ee.Initialize(credentials)

In [10]:
days_delta = 4
date = datetime.datetime.now() - datetime.timedelta(days=days_delta)
date = date.strftime('%Y_%m_%d')

try:
  current_time = datetime.datetime.now()

  Map = geemap.Map(center=[-32.7502,-70.4381], zoom=9)
  streamflow = ee.FeatureCollection("WWF/HydroSHEDS/v1/FreeFlowingRivers").filter(ee.Filter.eq('BAS_ID', 3975615)).style(color= 'B2B2B3')
  aconcagua_cuenca = ee.FeatureCollection("users/observatorionieves/DGA/aconcagua_cuenca").style(color= '#FF0000', width=1, fillColor= '00000000')
  subsubcuencas_ms = ee.FeatureCollection("users/observatorionieves/DGA/aconcagua_subsubcuencas_ms")
  MODIScollection = ee.Image('MODIS/006/MOD10A1/' + date).select(['NDSI_Snow_Cover'],['Snow_Albedo_Daily_Tile_Class']).clip(subsubcuencas_ms)

  colorizedVis = {
    'min': 0.0,
    'max': 100.0,
    'palette': ['#ffffff','#ffffd9','#edf8b1','#c7e9b4','#7fcdbb','#41b6c4','#1d91c0','#225ea8','#253494','#081d58']
  }
  Map.add_basemap('SATELLITE')
  Map.addLayer(MODIScollection,colorizedVis,"NDSI promedio") 
  Map.addLayer(streamflow,{},'Red hídrica') 
  Map.addLayer(aconcagua_cuenca,{},'Cuenca Río Aconcagua')
  Map.addLayer(subsubcuencas_ms.style(fillColor= '00000000',color= '#000000', width= 1),{},'Selección media-superior cuenca del Río Aconcagua')
  # Map
  
  # Generar HTML
  production = False
  if production:
      download_dir_base = 'web/media/maps/modis/doy/terra'
  else:
      download_dir_base = 'Downloads/maps/modis/doy/terra'
  
  html_file_name = date + '.html'
  title_html = 'MODIS Terra DOY - '+ date
  output = f'{current_time}: MODIS Terra DOY {date}'

  download_dir = os.path.join(os.path.expanduser('~'), download_dir_base)
  if not os.path.exists(download_dir):
      os.makedirs(download_dir)
  html_file = os.path.join(download_dir, html_file_name)

  Map.to_html(outfile=html_file, title=title_html, width='100%', height='880px')
  print(output)
except Exception as e:
  print(e)



2022-05-17 14:51:10.132398: MODIS Terra DOY 2022_05_13


In [5]:
MODIScollection_2 = ee.ImageCollection('MODIS/006/MOD10A1').select('NDSI_Snow_Cover')
# print(MODIScollection_2.aggregate_array('system:id').getInfo())